In [10]:
import pickle
import pandas as pd
from prophet import Prophet

# Функция для загрузки моделей из файла
def load_models(file_path='fitted_models.pkl'):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

# Функция для получения прогноза от SARIMA модели
def sarima_predict(sarima_model, steps=31):
    forecast = sarima_model.forecast(steps=steps)
    return forecast

# Функция для получения прогноза от Prophet модели
def prophet_predict(prophet_model, future_df):
    forecast = prophet_model.predict(future_df)
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

# Основная функция для инференса
def inference_from_models(models, region_name, steps=31):
    # Извлечь модели для конкретного региона
    region_models = models.get(region_name)
    if not region_models:
        raise ValueError(f"No models found for region: {region_name}")

    # SARIMA Inference
    sarima_model = region_models['sarima_model']
    sarima_forecast = sarima_predict(sarima_model, steps=steps)

    # Prophet Inference
    prophet_model = region_models['prophet_model']
    
    # Создание будущего датафрейма для прогноза Prophet
    future_dates = sarima_forecast.index
    future_df = pd.DataFrame({'ds': future_dates})
    future_df['sarima_regressor'] = list(sarima_forecast)
    #future['sarima_regressor1'] = list(sarima_forecast11)
    future_df['myatezh_prigozhina'] = 0
    # Выполнить прогноз с использованием модели Prophet
    prophet_forecast = prophet_predict(prophet_model, future_df)

    return prophet_forecast


In [11]:
# Display forecasts for all regions
# Загружаем сохраненные модели
models = load_models('fitted_models.pkl')
regs = regions_data = [
    'Moscow',
    'Krasnodar',
    'St_Petersburg',
    'Moscow_Region',
    'Tatarstan',
    'Sverdlovsk',
    'Rostov',
    'Nizhny_Novgorod',
    'Another Not Russia',
    'Another Russia1',
]

# Задаем регион и количество шагов для прогноза
steps = 31
all_forecast = pd.DataFrame()
for region_name in regs:
    # Выполняем инференс (прогноз) для конкретного региона
    prophet_forecast = inference_from_models(models, region_name, steps).reset_index()
    all_forecast = pd.concat([all_forecast, prophet_forecast])

In [12]:
ans = all_forecast[['ds', 'yhat']].groupby('ds').sum().reset_index().rename({'ds': 'date', 'yhat': 'forecast_value'}, axis=1)
ans

,date,forecast_value
0,2024-07-01,8.600446e+07
1,2024-07-02,8.615935e+07
2,2024-07-03,8.321049e+07
3,2024-07-04,8.524843e+07
4,2024-07-05,8.551517e+07
5,2024-07-06,4.746329e+07
6,2024-07-07,4.773674e+07
7,2024-07-08,8.738544e+07
8,2024-07-09,8.793646e+07
9,2024-07-10,8.483973e+07
